In [1]:
# import all packages
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules
from mlxtend.preprocessing import TransactionEncoder

# set display options
%matplotlib inline
pd.set_option('display.max_rows', 200)
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_colwidth', 100)

# set style and palette
sns.set_theme(style = 'dark')

In [2]:
# load datasets
products = pd.read_csv('products.csv')
aisles = pd.read_csv('aisles.csv')
departments = pd.read_csv('departments.csv')
orders = pd.read_csv('orders.csv')
order_details = pd.read_csv('order_products_train.csv')

In [3]:
products.head()

,product_id,product_name,aisle_id,department_id
0,1,Chocolate Sandwich Cookies,61,19
1,2,All-Seasons Salt,104,13
2,3,Robust Golden Unsweetened Oolong Tea,94,7
3,4,Smart Ones Classic Favorites Mini Rigatoni With Vodka Cream Sauce,38,1
4,5,Green Chile Anytime Sauce,5,13


In [4]:
products.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 49688 entries, 0 to 49687
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   product_id     49688 non-null  int64 
 1   product_name   49688 non-null  object
 2   aisle_id       49688 non-null  int64 
 3   department_id  49688 non-null  int64 
dtypes: int64(3), object(1)
memory usage: 1.5+ MB


In [5]:
aisles.head()

,aisle_id,aisle
0,1,prepared soups salads
1,2,specialty cheeses
2,3,energy granola bars
3,4,instant foods
4,5,marinades meat preparation


In [6]:
aisles.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 134 entries, 0 to 133
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   aisle_id  134 non-null    int64 
 1   aisle     134 non-null    object
dtypes: int64(1), object(1)
memory usage: 2.2+ KB


In [7]:
departments.head()

,department_id,department
0,1,frozen
1,2,other
2,3,bakery
3,4,produce
4,5,alcohol


In [8]:
departments.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21 entries, 0 to 20
Data columns (total 2 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   department_id  21 non-null     int64 
 1   department     21 non-null     object
dtypes: int64(1), object(1)
memory usage: 468.0+ bytes


In [9]:
orders.head()

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
0,2539329,1,prior,1,2,8,NaN
1,2398795,1,prior,2,3,7,15.0
2,473747,1,prior,3,3,12,21.0
3,2254736,1,prior,4,4,7,29.0
4,431534,1,prior,5,4,15,28.0


In [10]:
orders.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3421083 entries, 0 to 3421082
Data columns (total 7 columns):
 #   Column                  Dtype  
---  ------                  -----  
 0   order_id                int64  
 1   user_id                 int64  
 2   eval_set                object 
 3   order_number            int64  
 4   order_dow               int64  
 5   order_hour_of_day       int64  
 6   days_since_prior_order  float64
dtypes: float64(1), int64(5), object(1)
memory usage: 182.7+ MB


In [11]:
order_details.head()

,order_id,product_id,add_to_cart_order,reordered
0,1,49302,1,1
1,1,11109,2,1
2,1,10246,3,0
3,1,49683,4,0
4,1,43633,5,1


In [12]:
order_details.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1384617 entries, 0 to 1384616
Data columns (total 4 columns):
 #   Column             Non-Null Count    Dtype
---  ------             --------------    -----
 0   order_id           1384617 non-null  int64
 1   product_id         1384617 non-null  int64
 2   add_to_cart_order  1384617 non-null  int64
 3   reordered          1384617 non-null  int64
dtypes: int64(4)
memory usage: 42.3 MB


In [13]:
# merge order and product info

# consolidate product info
product_info = products.merge(aisles, on = 'aisle_id', how = 'left').merge(departments, on = 'department_id', how = 'left')
product_info.drop(columns = ['aisle_id','department_id'], inplace = True)

# use a subset of orders for runtime improvements
orders = orders[orders['eval_set'] == 'train']

# consolidate order info and merge with product info
df = orders.merge(order_details, on = 'order_id', how = 'inner').merge(product_info, on = 'product_id', how = 'inner')

# drop and reorder columns
new_column_order = ['user_id', 'order_id', 'product_id', 'product_name', 'aisle', 'department', 'add_to_cart_order', 'reordered', 'order_number', 'order_dow', 'order_hour_of_day', 'days_since_prior_order']
df = df[new_column_order]

# sort rows and reset index
df = df.sort_values(by = ['user_id', 'order_id', 'add_to_cart_order']).reset_index(drop = True)

In [14]:
df.head()

,user_id,order_id,product_id,product_name,aisle,department,add_to_cart_order,reordered,order_number,order_dow,order_hour_of_day,days_since_prior_order
0,1,1187899,196,Soda,soft drinks,beverages,1,1,11,4,8,14.0
1,1,1187899,25133,Organic String Cheese,packaged cheese,dairy eggs,2,1,11,4,8,14.0
2,1,1187899,38928,0% Greek Strained Yogurt,yogurt,dairy eggs,3,1,11,4,8,14.0
3,1,1187899,26405,XL Pick-A-Size Paper Towel Rolls,paper goods,household,4,1,11,4,8,14.0
4,1,1187899,39657,Milk Chocolate Almonds,candy chocolate,snacks,5,1,11,4,8,14.0


In [15]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1384617 entries, 0 to 1384616
Data columns (total 12 columns):
 #   Column                  Non-Null Count    Dtype  
---  ------                  --------------    -----  
 0   user_id                 1384617 non-null  int64  
 1   order_id                1384617 non-null  int64  
 2   product_id              1384617 non-null  int64  
 3   product_name            1384617 non-null  object 
 4   aisle                   1384617 non-null  object 
 5   department              1384617 non-null  object 
 6   add_to_cart_order       1384617 non-null  int64  
 7   reordered               1384617 non-null  int64  
 8   order_number            1384617 non-null  int64  
 9   order_dow               1384617 non-null  int64  
 10  order_hour_of_day       1384617 non-null  int64  
 11  days_since_prior_order  1384617 non-null  float64
dtypes: float64(1), int64(8), object(3)
memory usage: 126.8+ MB


In [16]:
# check for duplicates
# print(df.duplicated().any())

# convert dtypes
df['days_since_prior_order'] = df['days_since_prior_order'].astype('int')

# clean product_name
df['product_name'] = df['product_name'].str.replace(r'[^\w\s%#]', '', regex = True).str.strip().str.lower()

In [17]:
# preprocessing using transaction encoder

# string of items per order
df2 = df.groupby('order_id', as_index = False)['product_name'].apply(','.join)

# convert into python list of list
l = list(df2['product_name'].apply(lambda x: x.split(',')))

# instantiate transaction encoder object
encoder = TransactionEncoder()

# fit and transform
l2 = encoder.fit_transform(l)

# convert one-hot encoded data to dataframe
df3 = pd.DataFrame(l2, columns = encoder.columns_)
df3.head()

,#2 coffee filters,#2 cone white coffee filters,#2 mechanical pencils,#4 natural brown coffee filters,0 calorie acai raspberry water beverage,0 calorie fuji apple pear water beverage,0 calorie strawberry dragonfruit water beverage,0% fat black cherry greek yogurt y,0% fat blueberry greek yogurt,0% fat free organic milk,0% fat greek yogurt black cherry on the bottom,0% fat greek yogurt vanilla,0% fat organic greek vanilla yogurt,0% fat peach greek yogurt,0% fat strawberry greek yogurt,0% fat superfruits greek yogurt,0% fat vanilla greek yogurt,0% greek blueberry on the bottom yogurt,0% greek strained yogurt,0% greek yogurt black cherry on the bottom,0% milkfat greek plain yogurt,0% milkfat greek yogurt honey,1 % lowfat milk,1 apple 1 mango fruit bar,1 apple 1 pear fruit bar,1 liter,1 mg melatonin sublingual orange tablets,1 ply napkins,1 ply paper towels,1 razor handle and 2 freesia scented razor refills premium bladerazor system,1 step kashmir spinach indian cuisine,1 step1 minute noodles toasted sesame,1 to 1 gluten free baking flour,1% chocolate milk,1% hydrocortisone antiitch cream tube antiitch,1% low fat chocolate milk,1% low fat cottage cheese,1% low fat milk,1% lowfat chocolate milk,1% lowfat cottage cheese,1% lowfat cultured buttermilk,1% lowfat milk,1% lowfat organic milk,1% lowfat vanilla milk,1% milk,1% milkfat low fat buttermilk,1% milkfat low fat vitamin a d milk,1% milkfat lowfat cottage cheese,10 bamboo skewers,10 calorie ginger ale,...,zesty lemon hommus,zesty lemon hummus,zesty lemon quinoa blend,zesty lemon sorbet,zesty nacho cheese sauce,zesty nacho kale chips,zesty pizza sprouted flax snax,zesty ranch dressing,zesty salsa wheat thins crackers,zesty sriracha carrot hummus,zico natural pure premium coconut water,zinc sage shampoo,zinc 50 mg,zinc echinacea cherry lozenges,zinc elderberry and raspberry lozenges,zinc gluconate glycine natural flavor cherry lozenges,zinc target mins 50 mg gluten free tablets,zinfandel,zinfandel california 2010,zinfandel heritage,zinfandel lodi california,zinfandel wine,zinger tangerine orange caffeine free herbal tea,zingers cakes,zingers raspberry cakes,zingz queso fundido baked snack crackers,zipnsteam medium cooking bags,zipper bags sandwich 50 ct,zipper sandwich bags,ziti,ziti meatballs,ziti bolognese pasta bowl,ziti pasta,ziti rigate penne,ziti rigati,zoe organic evoo,zucchini banana amaranth organic baby food,zucchini gingerbread carrot smart cookies,zucchini noodles,zucchini pancakes,zucchini soufflé,zucchini soup,zucchini spinach banana blend veggies onthego stage 2 6 months and up,zucchini squash,zucchini squash baby courgette,zuzu luxe onyx mascara,zyflamend whole body,zzzquil liquid warming berry flavor sleepaid,zzzquil nighttime liquid sleepaid twin pack,zzzquil nighttime sleepaid liquicaps
0,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,

In [18]:
# find frequent itemsets with minimum support using apriori

frequent_itemsets = apriori(df3, min_support = 0.01, max_len = 3, use_colnames = True)
frequent_itemsets.sort_values('support', ascending = False, inplace = True, ignore_index = True)

# number of items in each frequent itemset
frequent_itemsets['no. of items'] = frequent_itemsets['itemsets'].apply(lambda x: len(x))
frequent_itemsets

,support,itemsets,no. of items
0,0.142719,(banana),1
1,0.117980,(bag of organic bananas),1
2,0.083028,(organic strawberries),1
3,0.074568,(organic baby spinach),1
4,0.062000,(large lemon),1
5,0.056467,(organic avocado),1
6,0.055583,(organic hass avocado),1
7,0.049494,(strawberries),1
8,0.045980,(limes),1
9,0.042268,(organic raspberries),1


In [19]:
# find association rules with minimum confidence

rules_1 = association_rules(frequent_itemsets, metric = "confidence", min_threshold = 0.1)
rules_1.sort_values(by = 'confidence', ascending = False, inplace = True, ignore_index = True)
rules_1

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
0,(organic hass avocado),(bag of organic bananas),0.055583,0.117980,0.018444,0.331825,2.812560,0.011886,1.320044,0.682381
1,(organic raspberries),(bag of organic bananas),0.042268,0.117980,0.013566,0.320952,2.720400,0.008579,1.298907,0.660318
2,(organic raspberries),(organic strawberries),0.042268,0.083028,0.012728,0.301118,3.626710,0.009218,1.312056,0.756233
3,(strawberries),(banana),0.049494,0.142719,0.014847,0.299969,2.101819,0.007783,1.224633,0.551518
4,(organic avocado),(banana),0.056467,0.142719,0.016889,0.299096,2.095698,0.008830,1.223107,0.554122
5,(organic strawberries),(bag of organic bananas),0.083028,0.117980,0.023428,0.282174,2.391714,0.013633,1.228738,0.634577
6,(large lemon),(banana),0.062000,0.142719,0.016447,0.265274,1.858714,0.007598,1.166803,0.492531
7,(limes),(large lemon),0.045980,0.062000,0.012156,0.264379,4.264159,0.009305,1.275113,0.802381
8,(organic baby spinach),(bag of organic bananas),0.074568,0.117980,0.017042,0.228536,1.937082,0.008244,1.143308,0.522739
9,(limes),(banana),0.045980,0.142719,0.010144,0.220620,1.545836,0.003582,1.099953,0.370119


In [20]:
# find association rules with minimum lift

rules_2 = association_rules(frequent_itemsets, metric = "lift", min_threshold = 1.5)
rules_2.sort_values(by = 'lift', ascending = False, inplace = True, ignore_index = True)
rules_2

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
0,(limes),(large lemon),0.045980,0.062000,0.012156,0.264379,4.264159,0.009305,1.275113,0.802381
1,(large lemon),(limes),0.062000,0.045980,0.012156,0.196066,4.264159,0.009305,1.186690,0.816085
2,(organic raspberries),(organic strawberries),0.042268,0.083028,0.012728,0.301118,3.626710,0.009218,1.312056,0.756233
3,(organic strawberries),(organic raspberries),0.083028,0.042268,0.012728,0.153295,3.626710,0.009218,1.131128,0.789847
4,(organic avocado),(large lemon),0.056467,0.062000,0.010281,0.182076,2.936692,0.006780,1.146805,0.698948
5,(large lemon),(organic avocado),0.062000,0.056467,0.010281,0.165827,2.936692,0.006780,1.131099,0.703071
6,(organic hass avocado),(bag of organic bananas),0.055583,0.117980,0.018444,0.331825,2.812560,0.011886,1.320044,0.682381
7,(bag of organic bananas),(organic hass avocado),0.117980,0.055583,0.018444,0.156331,2.812560,0.011886,1.119416,0.730654
8,(organic raspberries),(bag of organic bananas),0.042268,0.117980,0.013566,0.320952,2.720400,0.008579,1.298907,0.660318
9,(bag of organic bananas),(organic raspberries),0.117980,0.042268,0.013566,0.114987,2.720400,0.008579,1.082167,0.716998
